In [1]:
import numpy as np
import rasterio
import geopandas as gpd
from rasterio.enums import Resampling
from rasterio.transform import from_origin
from rasterio.mask import mask
import geopandas as gpd
from osgeo import gdal, gdalconst
import xdem
import matplotlib.pyplot as plt

In [2]:
#load with rasterio
with rasterio.open('./data/DEMs/Silvretta-2020-2m.tif') as dem_src:
    dem2020 = dem_src.read(1)

In [ ]:
# Explore metadata
# What is the file resolution and what is the coordinate system?
dem_src.meta

In [ ]:
dem_src.bounds

In [ ]:
# Plot elevation
f, ax = plt.subplots(figsize=(8,8))
cax = ax.imshow(dem2020,
           vmin=2000, 
           vmax=3300, 
           cmap='terrain')
cbar = f.colorbar(cax, ax=ax, label='Elevation', shrink=0.5)
ax.set_title('Silvretta elevation')
plt.show()

In [ ]:
# Plot elevation
# Get the bounds of the raster (leave this for participants to figure out, i.e. how to set this exactly).
bounds = dem_src.bounds
extent = [bounds.left, bounds.right, bounds.bottom, bounds.top]

In [ ]:
#mask no-data values for prettyness:
if dem_src.nodata is not None:
    dem2020 = np.ma.masked_equal(dem2020, dem_src.nodata)

In [ ]:
f, ax = plt.subplots(figsize=(8,8))
cax = ax.imshow(dem2020,
           extent=extent,
           vmin=2000, 
           vmax=3300, 
           cmap='terrain')
cbar = f.colorbar(cax, ax=ax, label='Elevation', shrink=0.5)
ax.set_title('Silvretta elevation')
plt.show()

In [3]:
dem = xdem.DEM('./data/DEMs/Silvretta-2020-2m.tif')

In [ ]:
dem.plot() #notice how it already knows where it is in the world

In [ ]:
xdem.terrain.hillshade(dem).plot(cmap='Greys_r')

In [ ]:
hillshade = xdem.terrain.hillshade(dem) #rendering on the fly is slow, let's save to a variable

In [4]:
rgi_thickness = xdem.DEM('./data/RGI60-11/RGI60-11.00804_thickness.tif')

Problem: ice thickness and our points are in different coordinate systems. We know how to reproject vector data,
so let's reproject the raster dataset instead.

In [5]:
# with xdem, reprojection is easy! 
thickness_lv95 = rgi_thickness.reproject(dem)

In [6]:
# load our outline 
outline = gpd.read_file('./data/silvretta_outline.geojson')

In [ ]:
f, ax = plt.subplots()
hillshade.plot(ax=ax,
               cmap='Greys_r',
               add_cbar=False,
               alpha=1.0)

thickness_lv95.plot(ax=ax, 
              cmap='viridis', 
              alpha=0.6)
outline.plot(ax=ax, facecolor='none', edgecolor='orange')
ax.set_xlim([outline.bounds.minx[0], outline.bounds.maxx[0]])
ax.set_ylim([outline.bounds.miny[0], outline.bounds.maxy[0]])

In [66]:
# sampling a raster dataset at predefined points or extracting information inside a 
# load geojson
sampling_points = gpd.read_file('./data/sampling_points.geojson')

In [8]:
thickness_rio = thickness_lv95.to_rio_dataset()

In [9]:
thickness_rio.nodata

-99999.0

In [10]:
masked_data, masked_transform = mask(thickness_rio, outline.geometry, crop=True, nodata=thickness_rio.nodata)

In [11]:
# Exclude NaN values from the masked array
masked_data = np.ma.masked_equal(masked_data, thickness_rio.nodata)

In [13]:
stats = {
    'min': masked_data.min(),
    'max': masked_data.max(),
    'mean': masked_data.mean(),
    'median': np.ma.median(masked_data),
    'std': masked_data.std()
}

print("Zonal Statistics:")
for key, value in stats.items():
    print(f"{key}: {value}")

Zonal Statistics:
min: 0.0
max: 153.71749877929688
mean: 77.89014095920692
median: 79.30809783935547
std: 24.415327814558005


In [ ]:
# Can you calculate the volume of ice inside the outline?

In [67]:
# Create a list to store raster values at points
raster_values_at_points = []

# Sample raster at each point
for index, point in sampling_points.iterrows():
    # Convert point coordinates to pixel coordinates
    pixel_coords = rasterio.transform.rowcol(thickness_rio.transform, point.geometry.x, point.geometry.y)

    # Get raster value at the pixel coordinate
    value = thickness_rio.read(1)[pixel_coords]

    # Add value to the list
    raster_values_at_points.append(value)

# Add raster values to GeoDataFrame
sampling_points['ice_thickness'] = raster_values_at_points

print(sampling_points)

   Points                         geometry  ice_thickness
0  Point1  POINT (2800379.700 1192725.200)      87.074844
1  Point2  POINT (2801251.500 1192437.900)     105.058449
2  Point3  POINT (2802045.600 1191907.400)     125.354164


### Now we want to get stats for a series of polygons. Let's say we don't trust the individual point measurements, so we put a buffer around our initial points

In [74]:
sampling_points['buffered']= sampling_points.geometry.buffer(100)

In [75]:
buffered_points = sampling_points.copy()
buffered_points = buffered_points.drop(['geometry','ice_thickness'], axis=1)

In [78]:
buffered_points

,Points,buffered
0,Point1,"POLYGON ((2800479.700 1192725.200, 2800479.218..."
1,Point2,"POLYGON ((2801351.500 1192437.900, 2801351.018..."
2,Point3,"POLYGON ((2802145.600 1191907.400, 2802145.118..."


In [77]:
buffered_points = buffered_points.set_geometry('buffered')

In [79]:
raster_stats_for_polygons = []

for index, row in sampling_points.iterrows():
    # Access the geometry of the current row
    #polygon = sampling_points.geometry[i]
    
    masked_data, masked_transform = mask(thickness_rio, [row.buffered], crop=True, nodata=thickness_rio.nodata)
    
    # Exclude nodata values from the masked array
    masked_data = np.ma.masked_equal(masked_data, thickness_rio.nodata)
    
    # Calculate statistics of the masked array
    stats = {
        'min': masked_data.min(),
        'max': masked_data.max(),
        'mean': masked_data.mean(),
        'median': np.ma.median(masked_data),
        'std': masked_data.std()
    }
    
    # Add statistics to the list
    raster_stats_for_polygons.append(stats)

# Add raster statistics to GeoDataFrame
buffered_points = buffered_points.assign(**{key: [stat[key] for stat in raster_stats_for_polygons] for key in raster_stats_for_polygons[0].keys()})

print(buffered_points)

   Points                                           buffered         min  \
0  Point1  POLYGON ((2800479.700 1192725.200, 2800479.218...   73.778915   
1  Point2  POLYGON ((2801351.500 1192437.900, 2801351.018...   90.493332   
2  Point3  POLYGON ((2802145.600 1191907.400, 2802145.118...  107.086716   

          max        mean      median       std  
0   94.967384   85.350885   85.795532  5.300182  
1  114.209724  104.534113  105.653465  4.915701  
2  134.701660  122.876816  123.341003  6.527680  


In [ ]:
# loading a second DEM and differencing them

In [ ]:
dem2014 = xdem.DEM('./data/DEMs/2014_SILVRETTA_DSM_1m_LV95_LN02_CIR_low_raw.tif')

In [ ]:
dem2014_reproj = dem2014.reproject(dem)

In [ ]:
diff = dem2014_reproj-dem

In [ ]:
f, ax = plt.subplots()
diff.plot(ax=ax, cmap='bwr', vmin=-10, vmax=10)
sgi.plot(ax=ax, facecolor='none')
ax.set_xlim([2795000, 2811000])
ax.set_ylim([1186250, 1195000])

In [ ]:
sgi = gpd.read_file('data/sgi_2016/SGI_2016_glaciers.shp')